# 패키지 로딩 및 환경 준비

In [1]:
# 필요한 기본 패키지 준비
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rc, font_manager
import datetime as dt

# 한글 처리
font_name = font_manager.FontProperties(fname='C:/Windows/Fonts/NanumGothicCoding.ttf').get_name()
rc('font',family=font_name)

# - 마이너스 사인 처리
matplotlib.rcParams['axes.unicode_minus'] = False

# jupyter notebook에서 warning 무시하기
import warnings
warnings.filterwarnings("ignore")

# 데이터 로딩 및 열(column) 순서 정렬

In [2]:
# 데이터 불러오기
gs = pd.read_csv('E:/Python/csv/korea_cvs.csv')
lavla = pd.read_csv('E:/Python/csv/korea_hnb.csv')
weather = pd.read_csv('E:/Python/csv/weather.csv', encoding='euc-kr')

In [3]:
weather.head(2)

,loc,date,temp,rain,cloud,wind,humid,hpa,sun_time,lgt_time,snow
0,108,2016-01-01,1.2,0.0,7.0,1.6,73.0,1019.9,9.6,2.1,0.0
1,108,2016-01-02,5.7,0.0,7.8,2.0,76.9,1012.0,9.6,3.6,0.0


In [4]:
# 컬럼네임 짧게 변형
gs.columns = ['pvn_nm','sale_dt','gender','age_cd','category','qty','bor_nm']
lavla.columns = ['pvn_nm','bor_nm','sale_dt','gender','age_cd','category','qty']

# 개인적인 편의를 위해 열(column)순서 변경
# gs및 lavla의 column순서가 동일하게 됨.
# 시, 구, 구매일, 성별, 나이, 물품, 수량 순으로 변경
gs = gs.iloc[:,[0,6,1,2,3,4,5]]

# 내가 원하는 순서대로 변경됨.

In [5]:
# 날씨데이터 정제

# 참고 : 날씨데이터에는 날짜가 string객체, yyyy-mm-dd형태로 들어와 있음
# 년/월/일/요일 추가
# 향후 작업을 편히 하기 위함, year, month, day 컬럼 생성

# 그냥 아무생각없이 int32타입으로 함. 원한다면 다른타입으로해도 상관없음
#   주의 할점은, 여기서 str으로 했다면 전부str, int로 했다면 전부int, datetime이면 전부 datetime으로
# gs편의점이나 랄라블라의 데이터셋에서도 같은 타입으로 설정 해주어야 편해진다.
weather['year'] = weather['date'].str.split('-',expand=True)[0].astype('int32')
weather['month'] = weather['date'].str.split('-',expand=True)[1].astype('int32')
weather['day'] = weather['date'].str.split('-',expand=True)[2].astype('int32')
weather['weekday']=pd.to_datetime(weather['date']).apply(pd.datetime.weekday)
# string 날짜를 pd.datetime으로 
weather['date'] = pd.to_datetime(weather['date'])
weather['nth_week'] = weather['date'].dt.week

# 데이터 전처리

In [6]:
# gs/lavla 날짜는 숫자로 들어와 있음

# gs데이터 정제 : 년/월/일/요일 추가
# 정수 나눗셈, 소숫점이하 버림 예) 20161225 // 10000 = 2016
gs['year'] = gs['sale_dt'] // 10000

# 년도를빼고 월일만 남겨둔뒤 월계산.
# 예) 20161212 -20160000 = 1225      1225 //100 = 12
gs['month'] = (gs['sale_dt'] - gs['year']*10000) // 100

# 년, 월 빼서 일 계산 
# 20161212 -2016*10000 - 12*100 - 25
gs['day'] = gs['sale_dt'] - gs['year']*10000 - gs['month']*100

# datetime 타입의 자료로 변화해서 'date'컬럼에 저장
# 현시점에선 year, month, day 컬럼이 맨오른쪽 3번째 부터 쭈르륵 나열 되어있음
gs['date'] = pd.to_datetime(gs.iloc[:,-3:])

# 참고 : 요일은 월요일(0)~일요일(6)의 숫자로 표현됨
gs['weekday']=pd.to_datetime(gs['date']).apply(pd.datetime.weekday)

# 그 해를 시작하고 몇번째 주 인지 알려주는 컬럼 생성
gs['nth_week']= gs['date'].dt.week

In [8]:
# 랄라블라 데이터 정제 
lavla['year'] = lavla['sale_dt'] // 10000
lavla['month'] = (lavla['sale_dt'] - lavla['year']*10000) // 100
lavla['day'] = lavla['sale_dt'] - lavla['year']*10000 - lavla['month']*100
lavla['date'] = pd.to_datetime(lavla.iloc[:,-3:])
lavla['weekday']=pd.to_datetime(lavla['date']).apply(pd.datetime.weekday)
lavla['nth_week'] = lavla['date'].dt.week

In [9]:
# 부산(143) 2017/07/29일의
# 구름량(8초과), 비오는날(강수량 0초과), 습도(80초과), 기압(1002초과), 여름계절(6~9월달) 조건과 비슷한
# 부산 지역의 2017/07월 평균 값으로 대체하기 위한 평균 기온 계산
weather.loc[(weather['cloud']>8) 
            & (weather['rain']>0) 
            & (weather['humid']>80) 
            & (weather['hpa']>1002)
            & (weather['month'].between(6,9, inclusive=True))
            & (weather['loc']==143), ['loc','temp','year','month']].groupby(['loc','year','month']).mean()

## 결측치 처리

In [10]:
# 부산 지역의 7월 평균 값으로 대체
# 25.96 약 26도
# 3683은 사전에 결측치로 확인해서 미리 찾아둠 행(row)위치 찾음.
weather.loc[3863,'temp'] = 26.0

# 강수량및 적설량의 결측치는 비/눈이 오지 않은것이라 판단 0으로 대체
weather.loc[weather['rain'].isna(),'rain'] = 0
weather.loc[weather['snow'].isna(),'snow'] = 0

# 일조시간은 극적인 변화를 보이지 않을 것이라 판단 결측치 바로 전날 값으로 채우기로 결정
weather['lgt_time']=weather['lgt_time'].fillna(method='ffill')
weather['sun_time']=weather['sun_time'].fillna(method='ffill')
# method='ffill'  : forward fill인가 그 비슷한 뜻으로 전값으로 넣겠다는 뜻

# 평균기압(hpa)은 극적인 변화를 보이지 않을 것이라 판단 결측치 바로 전날 값으로 채우기로 결정
weather['hpa']=weather['hpa'].fillna(method='ffill')

In [11]:
# 풍속(wind) 결측치가 포함된 달의 평균풍속으로 대체

# print('2017/10의 평균 풍속', weather.loc[(weather['loc']==108)&(weather['year']==2017) & (weather['month']==10), 'wind'].mean())
# 1.9366666666666663
weather.loc[(weather['wind'].isna()) &(weather['year']==2017) & (weather['month']==10), 'wind'] = 1.9

# print('2017/12의 평균 풍속', weather.loc[(weather['loc']==108)&(weather['year']==2017) & (weather['month']==12), 'wind'].mean() )
# 1.7827586206896548
weather.loc[(weather['wind'].isna()) &(weather['year']==2017) & (weather['month']==12), 'wind'] = 1.8

# print('2018/07의 평균 풍속', weather.loc[(weather['loc']==112)&(weather['year']==2018) & (weather['month']==7), 'wind'].mean() )
# 1.6903225806451612
weather.loc[(weather['wind'].isna()) & (weather['year']==2018) & (weather['month']==7), 'wind'] = 1.7

In [12]:
# 결측치 확인
weather.isnull().sum()
# 결측치 제거 확인

loc         0
date        0
temp        0
rain        0
cloud       0
wind        0
humid       0
hpa         0
sun_time    0
lgt_time    0
snow        0
dtype: int64

## 데이터 중간 저장

In [14]:
gs.to_csv('E:/Python/처리된 csv자료/p_gs.csv', index=False, encoding='utf-8')
lavla.to_csv('E:/Python/처리된 csv자료/p_lavla.csv', index=False, encoding='utf-8')
weather.to_csv('E:/Python/처리된 csv자료/p_weather.csv', index=False, encoding='utf-8')
print('saving done!')

saving done!


## 데이터 기본 정보 확인

In [15]:
gs.info() # 270만
lavla.info() # 82만
weather.info() # 7672

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2707786 entries, 0 to 2707785
Data columns (total 7 columns):
korea_cvs.pvn_nm      object
korea_cvs.sale_dt     int64
korea_cvs.gen_cd      object
korea_cvs.age_cd      object
korea_cvs.category    object
korea_cvs.adj_qty     int64
korea_cvs.bor_nm      object
dtypes: int64(2), object(5)
memory usage: 144.6+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 820274 entries, 0 to 820273
Data columns (total 7 columns):
korea_hnb.pvn_nm      820274 non-null object
korea_hnb.bor_nm      820274 non-null object
korea_hnb.sale_dt     820274 non-null int64
korea_hnb.gen_cd      820274 non-null object
korea_hnb.age_cd      820274 non-null object
korea_hnb.category    820274 non-null object
korea_hnb.qty         820274 non-null int64
dtypes: int64(2), object(5)
memory usage: 43.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7672 entries, 0 to 7671
Data columns (total 11 columns):
loc         7672 non-null int64
date        7672 non-n